In [1]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow

import pandas as pd
import numpy as np
from PIL import Image
from sklearn.preprocessing import OneHotEncoder

In [2]:
sagemaker.__version__

'2.59.1.post0'

## Session setting

In [2]:
boto_session = boto3.Session(profile_name='MY_PROFILE_NAME',region_name='REGION_NAME')
sess = sagemaker.Session(boto_session=boto_session)

In [3]:
arn = 'MY_ARN'

In [16]:
# when running with sagemaker notebook instance, using the folowing

# sagemaker_session = sagemaker.Session()
# role = get_execution_role(sess)

## Define the uris required for training

In [4]:
bucket = 'BUCKET_NAME'
train_prefix = r'PATH_TO_TRAINING_DATA'
model_prefix = r'PATH_TO_TRAINING_MODEL'
unique_images_prefix = r'PATH_TO_TRAINING_IMAGE_DATA'

training_uri = r's3://{}/{}'.format(bucket, train_prefix)
model_uri = r's3://{}/{}'.format(bucket, model_prefix)
unique_images_uri = r's3://{}/{}'.format(bucket, unique_images_prefix)

## Init a tensorflow container

In [5]:
# change this to actual machine type if on cloud instance
# instance_type = 'local'
instance_type = 'ml.m5.xlarge'

In [6]:
estimator = TensorFlow(
    entry_point='tf_complex_cnn.py', # the script of algorithm will be run in the training env
    role=arn, 
    instance_count=1,
    instance_type=instance_type, 
    framework_version='2.4', 
    py_version='py37', 
    output_path=model_uri, # s3 uri that the model artifact will be stored, will be pointed to model_dir in the training env
    base_job_name='tf-complex-cnn', 
    sagemaker_session=sess,
    distribution={'parameter_server': {'enabled': False}}
)

In [7]:
# specify one or more s3 uris pointing to the data taht will be use while training
# these are called channels and can be accessed with the env_var SM_CHANNEL_{KEY}
estimator.fit({
    'training': training_uri,
    'unique_images': unique_images_uri,
})

2021-09-13 17:32:00 Starting - Starting the training job...
2021-09-13 17:32:23 Starting - Launching requested ML instancesProfilerReport-1631554319: InProgress
...
2021-09-13 17:33:03 Starting - Preparing the instances for training......
2021-09-13 17:34:44 Downloading - Downloading input data
2021-09-13 17:34:44 Training - Downloading the training image...
2021-09-13 17:35:04 Training - Training image download completed. Training in progress.2021-09-13 17:35:00.176186: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-09-13 17:35:00.179623: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2021-09-13 17:35:00.284463: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-09-13 17:35:02,882 sagemaker-training-toolkit INFO     Imported framework 

#015 1/25 [>.............................] - ETA: 3s - loss: 0.7775 - mse: 1.4814#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 2/25 [=>............................] - ETA: 25s - loss: 0.7845 - mse: 1.3965#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 3/25 [==>...........................] - ETA: 24s - loss: 0.7991 - mse: 1.3940#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#01


2021-09-13 17:36:44 Uploading - Uploading generated training model
2021-09-13 17:36:44 Completed - Training job completed
ProfilerReport-1631554319: NoIssuesFound
Training seconds: 134
Billable seconds: 134


## Deploy 

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type=instance_type, endpoint_name='tf-complex-cnn')

## Deploy using model artifact

In [6]:
from sagemaker.tensorflow import TensorFlowModel

In [9]:
model_data = estimator.model_data

In [8]:
model = TensorFlowModel(model_data=model_data, role=arn, framework_version='2.4', sagemaker_session=sess)

In [9]:
predictor = model.deploy(initial_instance_count=1, instance_type=instance_type, endpoint_name='tf-complex-cnn')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


----!

## NOTE:
* This is a multi-inputs network
* Inputs are numpy arrays in different shapes
* The default input serializer is json format, which does not work with ndarray
* Tried using NumpySerializer and custom inference.py solutions but not really work
* The ultimate walkaround is to remain the serializer as the json default and pass the inputs as the following format:
    
    ```
    input_data = {
        'inputs': {
            'input_1': x1.tolist(),
            'input_2': x2.tolist()
        }
    }
    ```
    (x1, x2 are ndarrays)

## Prepare input data

In [ ]:
df = pd.read_csv('agg_revised.csv')

In [ ]:
ages = sorted(df['age'].unique())
genders = sorted(df['gender'].unique())
seasons = sorted(df['season'].unique())
ohe = OneHotEncoder(categories=[ages, genders, seasons], sparse=False, drop='first')
ohe.fit(df[['age', 'gender', 'season']])

In [ ]:
img_path = '50.png'
targets = df[df['url_encoded']==img_path]

In [ ]:
img = np.array(Image.open('unique_images/{}'.format(img_path)).resize((128,128)))
img_inputs = np.array([img for i in range(len(targets))])

brf_inputs = pd.DataFrame(ohe.transform(targets[['age', 'gender', 'season']]))
brf_inputs['n'] = targets['flight'].values
brf_inputs = brf_inputs.values

In [ ]:
input_data = {
    'inputs': {
        'input_1': img_inputs.tolist(), 
        'input_2': brf_inputs.tolist()
    }
}

In [ ]:
pred = predictor.predict(input_data)

In [ ]:
predictor.delete_endpoint()